<a href="https://colab.research.google.com/github/sojiseung/sjs0424/blob/main/m3_%EB%B6%84%EC%84%9D_%EB%9D%BC%EC%9D%B4%EB%B8%8C%EB%9F%AC%EB%A6%AC/pandas_%EC%8B%A4%EC%8A%B5_titanic2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[실습] titanc3.csv 파일로 생존자 예측 모델 생성을 위한 분석용 데이터 셋을 작성하세요.

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving titanic3.csv to titanic3.csv


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# 데이터 불러오기
# file_path = '/content/drive/MyDrive/kdt_240424/m3_분석라이브러리/pandas/dataset/mtcars.csv'
file_path = 'titanic3.csv'
df = pd.read_csv(file_path,index_col=0)

# 데이터 확인
print(df.head())

        survived                                             name     sex  \
pclass                                                                      
1              1                    Allen, Miss. Elisabeth Walton  female   
1              1                   Allison, Master. Hudson Trevor    male   
1              0                     Allison, Miss. Helen Loraine  female   
1              0             Allison, Mr. Hudson Joshua Creighton    male   
1              0  Allison, Mrs. Hudson J C (Bessie Waldo Daniels)  female   

          age  sibsp  parch  ticket      fare    cabin embarked boat   body  \
pclass                                                                        
1       29.00      0      0   24160  211.3375       B5        S    2    NaN   
1        0.92      1      2  113781  151.5500  C22 C26        S   11    NaN   
1        2.00      1      2  113781  151.5500  C22 C26        S  NaN    NaN   
1       30.00      1      2  113781  151.5500  C22 C26        S  

In [ ]:
df.head()

,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
pclass,,,,,,,,,,,,,
1,1,"Allen, Miss. Elisabeth Walton",female,29.00,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,"Allison, Master. Hudson Trevor",male,0.92,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
1,0,"Allison, Miss. Helen Loraine",female,2.00,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.00,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.00,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [ ]:
df['cabin'].groupby

pclass
1         B5
1    C22 C26
1    C22 C26
1    C22 C26
1    C22 C26
      ...   
3        NaN
3        NaN
3        NaN
3        NaN
3        NaN
Name: cabin, Length: 1309, dtype: object

In [ ]:
# 결측치 확인
print(df.isnull().sum())

survived        0
name            0
sex             0
age           263
sibsp           0
parch           0
ticket          0
fare            1
cabin        1014
embarked        2
boat          823
body         1188
home.dest     564
dtype: int64


In [ ]:
# 필요없는 열 삭제
df.drop(columns=['name','ticket','body','home.dest','boat'], inplace=True)

In [ ]:
df.head()

,survived,sex,age,sibsp,parch,fare,cabin,embarked
pclass,,,,,,,,
1,1,female,29.00,0,0,211.3375,B5,S
1,1,male,0.92,1,2,151.5500,C22 C26,S
1,0,female,2.00,1,2,151.5500,C22 C26,S
1,0,male,30.00,1,2,151.5500,C22 C26,S
1,0,female,25.00,1,2,151.5500,C22 C26,S


In [ ]:
# 결측치 처리
df['age'].fillna(df['age'].mean(), inplace=True)
df['fare'].fillna(df['fare'].mean(), inplace=True)
df['embarked'].fillna(df['embarked'].mode()[0], inplace=True)

df.drop(columns=['cabin'], inplace=True)

In [ ]:
df.head()

,survived,sex,age,sibsp,parch,fare,embarked
pclass,,,,,,,
1,1,female,29.00,0,0,211.3375,S
1,1,male,0.92,1,2,151.5500,S
1,0,female,2.00,1,2,151.5500,S
1,0,male,30.00,1,2,151.5500,S
1,0,female,25.00,1,2,151.5500,S


In [ ]:
# age_group을 6개 범주로 확대
bins = [0,5,12,18,35,60,100]
labels = ['Infant','Child','Teenager','Young Adult','Adult','Senior']
df['age_group'] = pd.cut(df['age'], bins=bins,labels=labels)
df.drop(columns=['age'], inplace=True)

In [ ]:
df.head()

,survived,sex,sibsp,parch,fare,embarked,age_group
pclass,,,,,,,
1,1,female,0,0,211.3375,S,Young Adult
1,1,male,1,2,151.5500,S,Infant
1,0,female,1,2,151.5500,S,Infant
1,0,male,1,2,151.5500,S,Young Adult
1,0,female,1,2,151.5500,S,Young Adult


In [ ]:
# sibsp와 parch 합쳐서 가족규모변수생성
df['family_size'] = df['sibsp'] + df['parch'] + 1 # 자기 자신 포함
df.drop(columns=['sibsp', 'parch'], inplace=True)

In [ ]:
df.head()

,survived,sex,fare,embarked,age_group,family_size
pclass,,,,,,
1,1,female,211.3375,S,Young Adult,1
1,1,male,151.5500,S,Infant,4
1,0,female,151.5500,S,Infant,4
1,0,male,151.5500,S,Young Adult,4
1,0,female,151.5500,S,Young Adult,4


In [ ]:
# fare를 3개 범주로 변환
fare_bins = [0,30,100,600]
fare_labels = ['Low','Medium','High']
df['fare_group'] = pd.cut(df['fare'], bins=fare_bins, labels=fare_labels)
df.drop(columns=['fare'], inplace=True)

In [ ]:
df.head()

,survived,sex,embarked,age_group,family_size,fare_group
pclass,,,,,,
1,1,female,S,Young Adult,1,High
1,1,male,S,Infant,4,High
1,0,female,S,Infant,4,High
1,0,male,S,Young Adult,4,High
1,0,female,S,Young Adult,4,High


In [ ]:
# 범주형 변수를 더미 변수로 변환할 열 목록
categorical_columns = ['age_group','fare_group','sex','embarked']

for column in categorical_columns:
  df=pd.get_dummies(df,columns=[column],drop_first=True)

In [ ]:
df.head()

,survived,family_size,age_group_Child,age_group_Teenager,age_group_Young Adult,age_group_Adult,age_group_Senior,fare_group_Medium,fare_group_High,sex_male,embarked_Q,embarked_S
pclass,,,,,,,,,,,,
1,1,1,False,False,True,False,False,False,True,False,False,True
1,1,4,False,False,False,False,False,False,True,True,False,True
1,0,4,False,False,False,False,False,False,True,False,False,True
1,0,4,False,False,True,False,False,False,True,True,False,True
1,0,4,False,False,True,False,False,False,True,False,False,True


In [ ]:
# 타겟 변수와 피처 변수 설정
X = df.drop('survived', axis=1)
y = df['survived']

# 학습 데이터와 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 데이터 크기 확인
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# 랜덤 포레스트 모델 학습
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 예측
y_pred = model.predict(X_test)

# 성능 평가
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


(1047, 11) (262, 11) (1047,) (262,)
Accuracy: 0.767175572519084
Confusion Matrix:
 [[123  21]
 [ 40  78]]
Classification Report:
               precision    recall  f1-score   support

           0       0.75      0.85      0.80       144
           1       0.79      0.66      0.72       118

    accuracy                           0.77       262
   macro avg       0.77      0.76      0.76       262
weighted avg       0.77      0.77      0.76       262



파생변수 : 기존 데이터로 만든 새로운 변수
- 각 주제 또는 부팀에서 데이터를 분석하기 위해 새로운 변수를 조합해서 사용함
- 이떄, 어떻게 변수를 만들어 내는지에 따라 주관적인 개념이 들어갈 수 있음
- 파생변수 생성시 주관적인 판단이 개입될 수 있어 논리적으로 타당한지 체크해야함
- 기존 변수의 연산, 조합, 분해 또는 함수, 조건문 등으로 데이터를 새로 생성함

Task1. train.csv 파일로 생존자 예측 모델 생성을 위한 분석용 데이터 셋을 작성하세요.

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

file_path = 'train.csv'
df = pd.read_csv(file_path,index_col=0)

# 데이터 확인
print(df.head())

             Survived  Pclass  \
PassengerId                     
1                   0       3   
2                   1       1   
3                   1       3   
4                   1       1   
5                   0       3   

                                                          Name     Sex   Age  \
PassengerId                                                                    
1                                      Braund, Mr. Owen Harris    male  22.0   
2            Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0   
3                                       Heikkinen, Miss. Laina  female  26.0   
4                 Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0   
5                                     Allen, Mr. William Henry    male  35.0   

             SibSp  Parch            Ticket     Fare Cabin Embarked  
PassengerId                                                          
1                1      0         A/5 21171   7.2500   NaN        S

In [ ]:
df.isnull().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age           0
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [ ]:
df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
df[df['Embarked'].isna()]

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN
830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN


In [ ]:
df.drop(columns=['Name','Ticket','Cabin'],inplace=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 891 entries, 1 to 891
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  891 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 62.6+ KB


In [ ]:
df['Embarked'].fillna(df['Embarked'].mode()[0],inplace=True)
df['Age'].fillna(df['Age'].mean(), inplace=True)

In [ ]:
df['family_size'] = df['SibSp'] + df['Parch'] + 1
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,family_size
PassengerId,,,,,,,,,
1,0,3,male,22.0,1,0,7.2500,S,2
2,1,1,female,38.0,1,0,71.2833,C,2
3,1,3,female,26.0,0,0,7.9250,S,1
4,1,1,female,35.0,1,0,53.1000,S,2
5,0,3,male,35.0,0,0,8.0500,S,1


In [ ]:
bins = [0,5,12,18,35,60,100]
labels = ['Infant','Child','Teenager','Young Adult','Adult','Senior']
df['age_group'] = pd.cut(df['Age'], bins=bins, labels=labels)
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,family_size,age_group
PassengerId,,,,,,,,,,
1,0,3,male,22.0,1,0,7.2500,S,2,Young Adult
2,1,1,female,38.0,1,0,71.2833,C,2,Adult
3,1,3,female,26.0,0,0,7.9250,S,1,Young Adult
4,1,1,female,35.0,1,0,53.1000,S,2,Young Adult
5,0,3,male,35.0,0,0,8.0500,S,1,Young Adult


In [ ]:
fare_bins = [0,50,250,600]
fare_labels = ['Low','Mideum','High']
df['fare_group'] = pd.cut(df['Fare'],bins=fare_bins,labels=fare_labels)
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,family_size,age_group,fare_group
PassengerId,,,,,,,,,,,
1,0,3,male,22.0,1,0,7.2500,S,2,Young Adult,Low
2,1,1,female,38.0,1,0,71.2833,C,2,Adult,Mideum
3,1,3,female,26.0,0,0,7.9250,S,1,Young Adult,Low
4,1,1,female,35.0,1,0,53.1000,S,2,Young Adult,Mideum
5,0,3,male,35.0,0,0,8.0500,S,1,Young Adult,Low


In [ ]:
# 생존자가 가족이 있는지 없는지 확인
df['Is_Alone'] = df['family_size'] <= 1
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,family_size,age_group,fare_group,Is_Alone
PassengerId,,,,,,,,,,,,
1,0,3,male,22.0,1,0,7.2500,S,2,Young Adult,Low,False
2,1,1,female,38.0,1,0,71.2833,C,2,Adult,Mideum,False
3,1,3,female,26.0,0,0,7.9250,S,1,Young Adult,Low,True
4,1,1,female,35.0,1,0,53.1000,S,2,Young Adult,Mideum,False
5,0,3,male,35.0,0,0,8.0500,S,1,Young Adult,Low,True


In [ ]:
class_bins = [0,1,2,3]
class_labels = ['HightClass','MideumClass','LowClass']
df['P_class'] = pd.cut(df['Pclass'], bins=class_bins, labels=class_labels)
df.drop(columns=['Pclass'],inplace=True)
df.head()

,Survived,Sex,Age,SibSp,Parch,Fare,Embarked,family_size,age_group,fare_group,Is_Alone,P_class
PassengerId,,,,,,,,,,,,
1,0,male,22.0,1,0,7.2500,S,2,Young Adult,Low,False,LowClass
2,1,female,38.0,1,0,71.2833,C,2,Adult,Mideum,False,HightClass
3,1,female,26.0,0,0,7.9250,S,1,Young Adult,Low,True,LowClass
4,1,female,35.0,1,0,53.1000,S,2,Young Adult,Mideum,False,HightClass
5,0,male,35.0,0,0,8.0500,S,1,Young Adult,Low,True,LowClass


In [ ]:
df.drop(columns=['Age','SibSp','Parch','Fare'], inplace=True)
df.head()

,Survived,Sex,Embarked,family_size,age_group,fare_group,Is_Alone,P_class
PassengerId,,,,,,,,
1,0,male,S,2,Young Adult,Low,False,LowClass
2,1,female,C,2,Adult,Mideum,False,HightClass
3,1,female,S,1,Young Adult,Low,True,LowClass
4,1,female,S,2,Young Adult,Mideum,False,HightClass
5,0,male,S,1,Young Adult,Low,True,LowClass


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 891 entries, 1 to 891
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   Survived     891 non-null    int64   
 1   Sex          891 non-null    object  
 2   Embarked     891 non-null    object  
 3   family_size  891 non-null    int64   
 4   age_group    891 non-null    category
 5   fare_group   876 non-null    category
 6   Is_Alone     891 non-null    bool    
 7   P_class      891 non-null    category
dtypes: bool(1), category(3), int64(2), object(2)
memory usage: 38.8+ KB


In [ ]:
df.head()

,Survived,Sex,Embarked,family_size,age_group,fare_group,Is_Alone,P_class
PassengerId,,,,,,,,
1,0,male,S,2,Young Adult,Low,False,LowClass
2,1,female,C,2,Adult,Mideum,False,HightClass
3,1,female,S,1,Young Adult,Low,True,LowClass
4,1,female,S,2,Young Adult,Mideum,False,HightClass
5,0,male,S,1,Young Adult,Low,True,LowClass


In [ ]:
category_columns = ['Sex','age_group','fare_group','P_class','Embarked']

for column in category_columns:
  df = pd.get_dummies(df, columns=[column])
df.head()

,Survived,family_size,Is_Alone,Sex_female,Sex_male,age_group_Infant,age_group_Child,age_group_Teenager,age_group_Young Adult,age_group_Adult,age_group_Senior,fare_group_Low,fare_group_Mideum,fare_group_High,P_class_HightClass,P_class_MideumClass,P_class_LowClass,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,,,,,,,,,,,,
1,0,2,False,False,True,False,False,False,True,False,False,True,False,False,False,False,True,False,False,True
2,1,2,False,True,False,False,False,False,False,True,False,False,True,False,True,False,False,True,False,False
3,1,1,True,True,False,False,False,False,True,False,False,True,False,False,False,False,True,False,False,True
4,1,2,False,True,False,False,False,False,True,False,False,False,True,False,True,False,False,False,False,True
5,0,1,True,False,True,False,False,False,True,False,False,True,False,False,False,False,True,False,False,True


In [ ]:
# 타겟 변수와 피처 변수 설정
X = df.drop('Survived', axis=1)
y = df['Survived']

# 학습 데이터와 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 데이터 크기 확인
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# 랜덤 포레스트 모델 학습
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 예측
y_pred = model.predict(X_test)

# 성능 평가
print("Accuracy:", accuracy_score(y_test, y_pred))
#print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
#print("Classification Report:\n", classification_report(y_test, y_pred))


(712, 19) (179, 19) (712,) (179,)
Accuracy: 0.8379888268156425
